In [1]:
import os
import logging

import trimesh

import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

# Calculation of visual fidelity metrics

One of the most inportant things is to somehow measure the visual resemblence of the original and simplified meshes. After some search with Google and the papers of [Garland and Heckbert (1997)](https://www.cs.cmu.edu/~garland/Papers/quadrics.pdf) and [Elena Ovreiu. Accurate 3D mesh simplification (2012)](https://theses.hal.science/tel-01224848/file/these.pdf), I chose three metrics (I could have gone with just one in the Garland and Heckbert paper, but all were not that difficult to implement using `KDTree`s).

For the KDtree construction we use a fixed size: accuracy vs performance trade-off. Also it is a kind of standardization and ensures reproducibility of the results of the notebook (I listned it the last lectures when it was streesed upon multiple times). The meshes have realtively a low number of vertices as a whole, so the calculations will be accurate enough.

In [2]:
SAMPLE_SIZE = 10000 # 

#### Hausdorff Distance
**Hausdorff Distance** is a metric that measures the greatest distance from a point in one set to the nearest point in another set. In the context of 3D meshes, it calculates the maximum distance between the vertices of the original mesh and the simplified mesh. This metric captures the worst-case error, reflecting how far the two meshes deviate from each other at their most distant points.

The **Hausdorff Distance** between two sets of points $A$ and $B$ is defined as:

\begin{equation*}
d_H(A, B) = \max\left\{\sup_{a \in A} \inf_{b \in B} \|a - b\|, \sup_{b \in B} \inf_{a \in A} \|b - a\|\right\}
\end{equation*}

In [3]:
def hausdorff_distance(mesh1, mesh2):
    """
    Calculate the Hausdorff Distance between two meshes.
    
    Parameters:
    mesh1 (trimesh.Trimesh): The first mesh, typically the original mesh.
    mesh2 (trimesh.Trimesh): The second mesh, typically the simplified mesh.

    Returns:
    float: The Hausdorff Distance between the two meshes.
    """
    # Sample points on the surface of each mesh
    points1 = mesh1.sample(SAMPLE_SIZE)
    points2 = mesh2.sample(SAMPLE_SIZE)
    
    # Create KD-trees for fast nearest neighbor search
    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)
    
    # Compute distances from each point in mesh1 to the nearest point in mesh2
    distances_1_to_2, _ = tree2.query(points1, k=1)
    
    # Compute distances from each point in mesh2 to the nearest point in mesh1
    distances_2_to_1, _ = tree1.query(points2, k=1)
    
    # The Hausdorff distance is the maximum of these distances
    hausdorff_dist = max(distances_1_to_2.max(), distances_2_to_1.max())
    return hausdorff_dist

#### RMSE (Root Mean Square Error)

**RMSE (Root Mean Square Error)** is a standard metric used to measure the average magnitude of the error between predicted and actual values. For 3D meshes, RMSE quantifies the average distance between corresponding vertices of the original and simplified meshes. It provides an overall measure of how well the simplified mesh approximates the original mesh, with lower RMSE values indicating a closer fit.

The **RMSE** between two sets of corresponding points $A = \{a_1, a_2, \dots, a_n\}$ representing the vertices of the original mesh and $B = \{b_1, b_2, \dots, b_n\}$ representing the vertices of the simplified mesh is defined as:

\begin{equation*}
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \|a_i - b_i\|^2}
\end{equation*}

In [4]:
def rmse(mesh1, mesh2):
    """
    Calculate the RMSE (Root Mean Square Error) between two meshes.
    This measures how well the simplified mesh (mesh2) approximates the original mesh (mesh1).
    
    Parameters:
    mesh1 (trimesh.Trimesh): The first mesh, the original mesh.
    mesh2 (trimesh.Trimesh): The second mesh, the simplified mesh.

    Returns:
    float: The RMSE between the two meshes.
    """
    # Sample points on the surface of each mesh
    points1 = mesh1.sample(SAMPLE_SIZE)
    points2 = mesh2.sample(SAMPLE_SIZE)
    
    # Create a KD-tree for the original mesh points
    tree1 = cKDTree(points1)
    
    # Compute distances from each point in the simplified mesh (mesh2) to the nearest point in the original mesh (mesh1)
    distances, _ = tree1.query(points2, k=1)
    
    # Compute RMSE
    rmse_value = np.sqrt(np.mean(distances**2))
    return rmse_value


#### The metric described in Section 6.1 of the paper titled "Surface Simplification Using Quadric Error Metrics" by Michael Garland and Paul S. Heckbert
Measures the quality of approximations by calculating the average squared distance between the original model and its simplified version. This metric is closely related to what is known as the *mean squared error* but applied to 3D meshes.
The error metric $E_i$ is defined as follows:

\begin{equation*}
E_i = \frac{1}{|X_n| + |X_i|} \left( \sum_{v \in X_n} d^2(v, M_i) + \sum_{v \in X_i} d^2(v, M_n) \right)
\end{equation*}

Where:
- $X_n$ is a set of points sampled on the original model $M_n$.
- $X_i$ is a set of points sampled on the simplified model $M_i$.
- $d(v, M)$ is the minimum distance from a point $v$ to the closest face of the model $M$.

This metric averages the squared distances from the points on the original mesh to the simplified mesh and vice versa.



In [5]:
def mesh_simplification_error(mesh1, mesh2):
    """
    Calculate the error metric as described in Section 6.1.
    
    Parameters:
    mesh1 (trimesh.Trimesh): The first mesh, typically the original mesh.
    mesh2 (trimesh.Trimesh): The second mesh, typically the simplified mesh.

    Returns:
    float: The Section 6.1 defined metric distnace between the two meshes.
    """
    points1 = mesh1.sample(SAMPLE_SIZE)
    points2 = mesh2.sample(SAMPLE_SIZE)
    
    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)
    
    # Sum of squared distances from original mesh points to simplified mesh
    distances_1_to_2, _ = tree2.query(points1, k=1)
    error_1_to_2 = np.sum(distances_1_to_2**2)
    
    # Sum of squared distances from simplified mesh points to original mesh
    distances_2_to_1, _ = tree1.query(points2, k=1)
    error_2_to_1 = np.sum(distances_2_to_1**2)
    
    # Combined error
    error_metric = (error_1_to_2 + error_2_to_1) / (len(points1) + len(points2))
    
    return error_metric

In [6]:
def process_meshes(original_folder, simplified_folder):
    """
    Process and compare 3D meshes from two directories, calculating several error metrics.

    This function iterates over pairs of meshes from the original and simplified directories,
    calculates the Hausdorff Distance, RMSE, and Garland-Heckbert error between each pair,
    and returns a pandas dataframe with the results.

    Parameters:
    original_folder (str): Path to the folder containing the original meshes.
    simplified_folder (str): Path to the folder containing the simplified meshes.

    Returns:
    pandas.DataFrame: A DataFrame containing the visual fidelity results for each pairs of mesh file.
    """
    data = []

    original_files = {os.path.splitext(f)[0]: f for f in os.listdir(original_folder)}
    
    for prefix, original_filename in original_files.items():
        original_filepath = os.path.join(original_folder, original_filename)

        # Find the corresponding simplified files with the same prefix
        simplified_files = [f for f in os.listdir(simplified_folder) if f == original_filename]

        for simplified_filename in simplified_files:
            simplified_filepath = os.path.join(simplified_folder, simplified_filename)

            original_mesh = trimesh.load(original_filepath)
            simplified_mesh = trimesh.load(simplified_filepath)
            
            logging.info(f"Processing files: {original_filename}")
            hausdorff_dist = hausdorff_distance(original_mesh, simplified_mesh)
            rmse_value = rmse(original_mesh, simplified_mesh)
            garland_heckbert_error = mesh_simplification_error(original_mesh, simplified_mesh)
            logging.info(f"Done processing files: {original_filename}")
            data.append([prefix, hausdorff_dist, rmse_value, garland_heckbert_error])
            
    columns = ['file_id', 'hausdorff_distance', 'rmse', 'garland_heckbert_error']

    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
original_folder_path = './../../data/Thingi10K/raw_meshes/FilteredFiles'
simplified_folder_path = './../../data/simplified_output/'
output_folder_path = './../../data/csv_data/visual_fidelity_data/'

In [8]:
run_folder_names = ['output_t0.0_r0.5_p2000', 'output_t0.1_r0.5_p2000', 'output_t0.1_r0.9_p2000', 'output_t0.3_r0.9_p2000']

In [9]:
log_file = 'process_fidelity_log.log'
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [10]:
visual_fidelity_data = process_meshes(original_folder_path, simplified_folder_path + run_folder_names[0])

Looks good to me :)

In [11]:
visual_fidelity_data

,file_id,hausdorff_distance,rmse,garland_heckbert_error
0,100026,26.177169,1.041855,7.348570
1,100029,7.911350,0.888609,1.040496
2,100031,1.347594,0.388934,0.150414
3,100032,2.552619,0.843002,0.707361
4,100075,0.607961,0.095559,0.005549
...,...,...,...,...
695,200683,0.893891,0.266128,0.070870
696,200685,1.332271,0.419334,0.174624
697,200687,0.802102,0.263326,0.070241
698,200691,0.395717,0.122549,0.015314


In [12]:
visual_fidelity_data.to_csv(output_folder_path + run_folder_names[0] + '_vf.csv', index=False)

In [13]:
del visual_fidelity_data

In [14]:
for name in run_folder_names[1:]:
    visual_fidelity_data = process_meshes(original_folder_path, simplified_folder_path + name)
    visual_fidelity_data.to_csv(output_folder_path + name + '_vf.csv', index=False)
    del visual_fidelity_data